In [4]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.ensemble import IsolationForest
from pandas.testing import assert_frame_equal

FRAC = 0.25
log = pd.read_pickle('/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/behavior_log_pv_user_filter_enc_0.25.pkl')
def gen_intervals(user, user_log, groupby_sdt):
    sorted_log = user_log.sort_values(by = ['time_stamp'])
    if len(sorted_log) > 1:
        intervals = {}
        time1 = sorted_log.time_stamp.to_numpy()[:-1]
        time2= sorted_log.time_stamp.to_numpy()[1:]
        name = sorted_log[groupby_sdt].to_numpy()[:-1]
        diffs = time2 - time1 
        return pd.DataFrame({groupby_sdt: name, 'interval': diffs})

def applyParallel(dfGrouped, func, groupby_sdt):
    res = Parallel(n_jobs= -1)(delayed(func)(name, group, groupby_sdt) for name, group in tqdm(dfGrouped))
#log.iloc[:100].groupby('user').apply(gen_intervals)
    return pd.concat(res)
#删除异常值 用异常分数的0.5作为阀值
def iforest_remove_outliner(name, intervals_groupby, groupby_sdt):
    sll = intervals_groupby.interval.tolist()

    X = np.array(sll).reshape(len(sll),1)
    clf = IsolationForest(random_state=0).fit(X)
    predict_result=clf.predict(X)
    tl=[]
    for i in range(len(sll)):
        if predict_result[i] == 1:
            tl.append(sll[i])
    if len(tl) > 0:
        thresholds = pd.DataFrame({groupby_sdt : [name],'{}_threshold'.format(groupby_sdt):[max(tl)]})
        return thresholds

if __name__ == "__main__":
    cate_intervals = applyParallel(log.groupby('user'), gen_intervals, 'cate')
    cate_thresholds = applyParallel(cate_intervals.groupby('cate'), iforest_remove_outliner, 'cate')
    pd.to_pickle(cate_thresholds.set_index('cate'), '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/cate_intervals')
    brand_intervals = applyParallel(log.groupby('user'), gen_intervals, 'brand')
    brand_thresholds = applyParallel(brand_intervals.groupby('brand'), iforest_remove_outliner, 'brand')
    pd.to_pickle(brand_thresholds.set_index('brand'), '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/brand_intervals')

In [2]:
import numpy as np

In [25]:

click_offsets = np.array([8,3,3,9,5,6,9,6,9,1, 1])
session_idx_arr = np.array([1,8,5,6,9,4,3,2,1])
iters = np.array([0,1,2,3,4])
click_offsets[session_idx_arr[iters]]

array([3, 9, 6, 9, 1])

In [26]:
click_offsets[session_idx_arr[iters] + 1] - click_offsets[session_idx_arr[iters]]

array([ 0, -8,  3, -3,  0])

In [4]:
import pandas as pd
FRAC = 0.25
SESS_COUNT = 5

In [5]:
dnn_feature_columns = pd.read_pickle('/Users/yuxuanyang/Downloads/DSIN-master/model_input/brand_dsin_fd_' +
                                         str(FRAC) + '_' + str(SESS_COUNT) + '.pkl')

In [13]:
for i in dnn_feature_columns:
    #print(i)
    print(type(i))

<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.SparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSparseFeat'>
<class 'deepctr.feature_column.VarLenSpa

In [14]:
56*56

3136

In [15]:
56*28

1568

In [16]:
256/28

9.142857142857142

In [28]:
import pandas  as pd
import numpy as np

df =  pd.DataFrame(pd.DataFrame({'session_key': [1,2,2,3,3], 'item_key': [4,6,1,2,6], 'time_key': [1,2,4,5,6]}))
df

,session_key,item_key,time_key
0,1,4,1
1,2,6,2
2,2,1,4
3,3,2,5
4,3,6,6


In [23]:
offsets = np.zeros(df['session_key'].nunique() + 1, dtype=np.int32)
offsets[1:] = df.groupby('session_key').size().cumsum()

In [36]:
item_ids = df['item_key'].unique()  # type is numpy.ndarray
item2idx = pd.Series(data=np.arange(len(item_ids)),
                        index=item_ids)
# Build itemmap is a DataFrame that have 2 columns (self.item_key, 'item_idx)
itemmap = pd.DataFrame({'item_key': item_ids,
                        'item_idx': item2idx[item_ids].values})

In [56]:
session_idx_arr = np.arange(df['session_key'].nunique())
session_idx_arr

array([0, 1, 2])

In [55]:
click_offsets = offsets
click_offsets

array([0, 1, 3, 5], dtype=int32)

In [63]:
np.arange(2)[(click_offsets[session_idx_arr[np.arange(2)]+1]- click_offsets[session_idx_arr[np.arange(2)]]) <=1]

array([0])

In [58]:
click_offsets[session_idx_arr[np.arange(2)]+1]

array([1, 3], dtype=int32)

In [60]:
session_idx_arr[np.arange(2)]+1

array([1, 2])

In [3]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pyarrow
from pandas import Timestamp
import itertools
from datetime import datetime

In [4]:
# my credential file
#key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-f2e09e85c3b3.json"
key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-7129911936a3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
query= """
    select *
    from `encoded-adviser-131008.yang_20230130.20000users`
"""
query_job = client.query(query)  # Make an API request.
users = query_job.to_dataframe()
pd.to_pickle(users, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/sample_users.pkl')

In [6]:
query= """
    select *
    from `encoded-adviser-131008.yang_20230130.apps`
"""
query_job = client.query(query)  # Make an API request.
apps = query_job.to_dataframe()
pd.to_pickle(apps, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/apps.pkl')

In [6]:
query= """
    select *
    from `encoded-adviser-131008.yang_20230130.logs_8days_20000users` 
"""
query_job = client.query(query)  # Make an API request.
user_logs = query_job.to_dataframe()
pd.to_pickle(user_logs, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/sample_user_logs.pkl')

In [1]:
user_logs

NameError: name 'user_logs' is not defined